<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/speach-privacy/libritts-FL-meanWeights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `ml818` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

In [3]:
################################################################################
# 1. IMPORTS
################################################################################
import torch
import torchaudio
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    TrainingArguments,
    Trainer
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from collections import defaultdict

################################################################################
# 2. LOAD THE ORIGINAL DATASET
#    We'll just load the "train" split of the LibriTTS dev dataset (which is small).
################################################################################
dataset = load_dataset("HamdanXI/libritts_dev_dataset_gender")["train"]
# dataset columns: ["audio", "gender", "speaker_id", "text_original", ...]

print("Full dataset size:", len(dataset))
print("Columns:", dataset.column_names)
print("Unique speaker_ids:", len(set(dataset["speaker_id"])))

################################################################################
# 3. PICK 40 SPEAKERS: 20 MALE + 20 FEMALE
################################################################################
male_speakers_all = list({ex["speaker_id"] for ex in dataset if ex["gender"] == "M"})
female_speakers_all = list({ex["speaker_id"] for ex in dataset if ex["gender"] == "F"})

random.seed(42)
chosen_male = random.sample(male_speakers_all, 20)
chosen_female = random.sample(female_speakers_all, 20)

chosen_speakers = chosen_male + chosen_female
print(f"Chosen {len(chosen_male)} male speakers:   {chosen_male}")
print(f"Chosen {len(chosen_female)} female speakers: {chosen_female}")

################################################################################
# 4. FILTER DATASET TO ONLY THOSE 40 SPEAKERS
################################################################################
filtered_ds = dataset.filter(lambda x: x["speaker_id"] in chosen_speakers)

################################################################################
# 5. FOR EACH SPEAKER, PICK THE SINGLE LONGEST AUDIO SAMPLE
################################################################################
# We'll group rows by speaker_id, then pick the sample with the largest audio["array"] length.

samples_by_speaker = defaultdict(list)
for ex in filtered_ds:
    spk = ex["speaker_id"]
    samples_by_speaker[spk].append(ex)

speaker_to_longest_example = {}
for spk, rows in samples_by_speaker.items():
    # Sort by audio length descending, pick the top row
    rows_sorted = sorted(rows, key=lambda x: len(x["audio"]["array"]), reverse=True)
    speaker_to_longest_example[spk] = rows_sorted[0]

# We'll have exactly 40 entries here, one per speaker
print("Number of chosen speakers:", len(speaker_to_longest_example))

################################################################################
# 6. LOAD PROCESSOR
################################################################################
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

################################################################################
# 7. PREPARE FUNCTION (resample + tokenize)
################################################################################
def prepare_batch(batch):
    """
    1) Resample audio to 16k if needed.
    2) Extract raw (unpadded) input_values via processor.
    3) Tokenize text.
    """
    sr = batch["audio"]["sampling_rate"]
    audio_array = batch["audio"]["array"]
    wave_tensor = torch.tensor(audio_array, dtype=torch.float32)
    if sr != 16000:
        wave_tensor = torchaudio.functional.resample(wave_tensor, sr, 16000)
    wave_16k = wave_tensor.numpy()

    # Audio feature extraction
    audio_features = processor.feature_extractor(wave_16k, sampling_rate=16000)
    # Text tokenization
    text_tokens = processor.tokenizer(batch["text_original"] if batch["text_original"] else "")

    return {
        "speaker_id":   batch["speaker_id"],
        "gender":       batch["gender"],
        "text_original": batch["text_original"],
        "input_values": audio_features["input_values"][0],
        "labels":       text_tokens["input_ids"]
    }

################################################################################
# 8. MAP OUR 40 SAMPLES INTO A SMALL DATASET
################################################################################
longest_samples_list = []
for spk_id in speaker_to_longest_example:
    longest_samples_list.append(speaker_to_longest_example[spk_id])

# Create a small Dataset from these 40 examples
small_ds = Dataset.from_dict({
    "audio":         [ex["audio"] for ex in longest_samples_list],
    "speaker_id":    [ex["speaker_id"] for ex in longest_samples_list],
    "gender":        [ex["gender"] for ex in longest_samples_list],
    "text_original": [ex["text_original"] for ex in longest_samples_list],
})

# Now map: remove columns we don't need except "audio", "speaker_id", "gender", "text_original"
# but we do want to *add* input_values, labels
keep_cols = ["audio","speaker_id","gender","text_original","input_values","labels"]
small_ds = small_ds.map(
    prepare_batch,
    remove_columns=[col for col in small_ds.column_names if col not in keep_cols]
)

################################################################################
# 9. DATA COLLATOR
################################################################################
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_values = [f["input_values"] for f in features]
        labels       = [f["labels"]       for f in features]

        audio_batch = self.processor.feature_extractor.pad(
            {"input_values": input_values},
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": labels},
            padding=self.padding,
            return_tensors="pt",
        )

        labels_tensor = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100
        )

        return {
            "input_values": audio_batch["input_values"],
            "labels":       labels_tensor
        }

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

################################################################################
# 10. FUNCTION TO TRAIN ON SINGLE SAMPLE
################################################################################
def train_on_single_sample(
    sample,
    base_model_name="facebook/wav2vec2-base-960h",
    num_train_epochs=1,
    device="cuda" if torch.cuda.is_available() else "cpu"
):
    """
    Fine-tune on a single sample (dictionary with "input_values", "labels").
    Returns the fine-tuned model on CPU.
    """
    from datasets import Dataset

    single_ds = Dataset.from_dict({
        "input_values": [sample["input_values"]],
        "labels":       [sample["labels"]]
    })

    model = Wav2Vec2ForCTC.from_pretrained(base_model_name)
    model.to(device)
    # freeze feature encoder
    model.freeze_feature_encoder()

    training_args = TrainingArguments(
        output_dir="temp_model",
        per_device_train_batch_size=1,
        num_train_epochs=num_train_epochs,
        evaluation_strategy="no",
        logging_steps=5,
        learning_rate=1e-5,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=single_ds,
        tokenizer=processor.feature_extractor
    )

    trainer.train()
    model.to("cpu")
    return model

################################################################################
# 11. COMPUTE LAYERWISE DIFFERENCES
################################################################################
def compute_layerwise_diff(new_model, base_model, num_layers=12):
    """
    Compare new_model vs. base_model for each Wav2Vec2 encoder layer.
    Return a list of length `num_layers` with the mean absolute difference
    for each layer.
    """
    from collections import defaultdict
    layerwise_params = defaultdict(list)

    base_dict = dict(base_model.named_parameters())
    new_dict  = dict(new_model.named_parameters())

    for name, base_param in base_dict.items():
        # Look for encoder layers in the name
        if ("wav2vec2.encoder.layers" in name) and (name in new_dict):
            new_param = new_dict[name]
            parts = name.split(".")
            # e.g. name might be "wav2vec2.encoder.layers.0.attention..." -> parts[3] = '0'
            try:
                layer_idx = int(parts[3])
            except:
                continue

            if layer_idx < num_layers:
                diff = (new_param - base_param).detach().cpu().view(-1)
                layerwise_params[layer_idx].append(diff)

    layer_diffs = []
    for i in range(num_layers):
        if i in layerwise_params:
            diffs_cat = torch.cat(layerwise_params[i], dim=0)
            # We'll store the mean absolute difference
            mad = diffs_cat.abs().mean().item()
            layer_diffs.append(mad)
        else:
            layer_diffs.append(0.0)

    return layer_diffs

################################################################################
# 12. MAIN LOOP:
#     For each of the 40 samples -> Fine-tune -> Compare -> Collect differences
################################################################################
base_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").eval()
base_model.to("cpu")

all_rows = []
for i, sample in enumerate(small_ds):
    spk_id  = sample["speaker_id"]
    gender  = sample["gender"]
    text    = sample["text_original"]
    audio   = sample["audio"]  # dict with keys like "array", "sampling_rate"

    print(f"[{i+1}/40] Fine-tuning on speaker {spk_id} (gender={gender})")

    # Fine-tune
    new_model = train_on_single_sample(sample)

    # Compute layerwise difference (12 layers)
    layer_diffs = compute_layerwise_diff(new_model, base_model, num_layers=12)

    # Build a row dict: text_original, audio, speaker_id, gender, plus 12 columns
    row_dict = {
        "text_original": text,
        "audio": audio,   # you can keep the full audio dict
        "speaker_id": spk_id,
        "gender": gender,
    }
    # Add 12 columns for layer diffs
    for layer_idx in range(12):
        row_dict[f"layer_{layer_idx}_diff"] = layer_diffs[layer_idx]

    all_rows.append(row_dict)

    del new_model
    torch.cuda.empty_cache()

# Now we have 40 rows with all the columns we need.

################################################################################
# 13. CREATE A NEW DATASET AND PUSH TO HUB
################################################################################
final_dataset = Dataset.from_list(all_rows)

# Optional: Remove any columns you do not want. Here we keep them all:
# final_dataset = final_dataset.remove_columns([...])  # if needed

print("Final dataset format:")
print(final_dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/590 [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/336M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/470M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/376M [00:00<?, ?B/s]

...

Step,Training Loss


[40/40] Fine-tuning on speaker 2035 (gender=F)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-c1f7aab855d3>:197: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Final dataset format:
Dataset({
    features: ['text_original', 'audio', 'speaker_id', 'gender', 'layer_0_diff', 'layer_1_diff', 'layer_2_diff', 'layer_3_diff', 'layer_4_diff', 'layer_5_diff', 'layer_6_diff', 'layer_7_diff', 'layer_8_diff', 'layer_9_diff', 'layer_10_diff', 'layer_11_diff'],
    num_rows: 40
})


In [4]:
final_dataset.push_to_hub("HamdanXI/libritts_FL")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/HamdanXI/libritts_FL/commit/997124b0b6210ca9d81562dbdea8d65e5f604935', commit_message='Upload dataset', commit_description='', oid='997124b0b6210ca9d81562dbdea8d65e5f604935', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/HamdanXI/libritts_FL', endpoint='https://huggingface.co', repo_type='dataset', repo_id='HamdanXI/libritts_FL'), pr_revision=None, pr_num=None)